In [ ]:
import pandas as pd
import csv

# Read CSV file and skip the first row (header)
df = pd.read_csv('2020_pop.csv', skiprows=1)

# Extract columns of interest and create a copy of the subset
columns_of_interest = ['Geographic Area Name', 'Estimate!!SEX AND AGE!!Total population']
subset_df = df[columns_of_interest].copy()

# Extract 'County' and 'State' from the 'Geographic Area Name' column
subset_df.loc[:, 'County'] = subset_df['Geographic Area Name'].str.split(',', expand=True)[0].str.strip()
subset_df.loc[:, 'State'] = subset_df['Geographic Area Name'].str.split(',', expand=True)[1].str.strip()

# Select the final columns for the result
col = ['County', 'State', 'Estimate!!SEX AND AGE!!Total population']
res = subset_df[col]

# Rename the 'Estimate!!SEX AND AGE!!Total population' column to 'Population'
res = res.rename(columns={'Estimate!!SEX AND AGE!!Total population': 'Population'})

# Save the result to 'county_pop.csv' without the index
res.to_csv('county_pop.csv', index=False)

# Calculate and save the sum of population for each state to 'state_pop.csv'
state_population_sum = res.groupby('State')['Population'].sum().reset_index()
state_population_sum.to_csv('state_pop.csv', index=False)

# Read Excel file and skip the first 4 rows (header)
dff = pd.read_excel('county.xlsx', dtype={'State Code (FIPS)': str, 'County Code (FIPS)': str}, skiprows=4)

# Select columns of interest from the Excel file
colu = ['State Code (FIPS)', 'County Code (FIPS)', 'State Name', 'County Name']
df1 = dff[colu].copy()

# Combine 'State Code (FIPS)' and 'County Code (FIPS)' to create 'id'
df1['id'] = df1['State Code (FIPS)'] + df1['County Code (FIPS)']

# Rename columns for consistency
df1 = df1.rename(columns={'County Name': 'County', 'State Name': 'State'})

# Select the final columns for the result
col1 = ['id', 'State', 'County']
df3 = df1[col1]

# Merge county population data with additional county information
merged_df = pd.merge(df3, res, on=['County', 'State'], how='left')
merged_df = merged_df.dropna()
merged_df.to_csv('id_county_pop.csv', index=False)

# Select columns of interest from the Excel file
colu1 = ['State Code (FIPS)', 'State Name']
df2 = dff[colu1].copy()

# Rename columns for consistency
df2 = df2.rename(columns={'State Name': 'State', 'State Code (FIPS)': 'id'})

# Merge state population data with additional state information
m = pd.merge(state_population_sum, df2, on='State', how='left')
m = m.drop_duplicates()
m.to_csv('id_state_pop.csv', index=False)